**Computes simple modal share statistics**

**Input**: "sources/data.csv" csv using the following columns:
* `transportation_mode`: used mode of transport for the trip (see table below)
* `start_time`: datetime of begining of trip
* `end_time`: datetime of end of trip
* `trace_gps`: list of (lon, lat) tuples
* `longitude`, `latitude`: coords of starting point (also included as first point of `trace_gps`)

**Input**: "sources/paris.geojson" Shape representation of Paris, to filter results to the inner city

**Output**: JSON file "../static/data/modal_share_gps.json" modal share per transport mode:
```json
{
    "BIKE_TRIP":0.0538816023,
    "BUS":0.0159517901,
    "FOOT":0.6593406593,
    "HIGH_SPEED_TRAIN":0.0001772421,
    "MOTORCYCLE":0.0024813896,
    "PASSENGER_CAR":0.066111308,
    "RAIL_TRIP":0.193371145,
    "SCOOTER":0.003190358,
    "UNKNOWN":0.0054945055
}
```

Also displays a few graphs on the way, that should be exported as json for frontend

In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
df = pd.read_csv("sources/data.csv")

In [ ]:
df

In [ ]:
tr = {
-10 : "NOT_DEFINED",
0 : "UNKNOWN",
1 : "PASSENGER_CAR",
2 : "MOTORCYCLE",
3 : "HEAVY_DUTY_VEHICLE",
4 : "BUS",
5 : "COACH",
6 : "RAIL_TRIP",
7 : "BOAT_TRIP",
8 : "BIKE_TRIP",
9 : "PLANE",
10 : "SKI",
11 : "FOOT",
12 : "IDLE",
13 : "OTHER",
101 : "SCOOTER",
102 : "HIGH_SPEED_TRAIN"
}
df['transportation_mode_tr'] = df['transportation_mode'].apply(lambda x: tr[x])

In [ ]:
df.groupby(['transportation_mode_tr']).size().plot.pie(autopct='%1.1f%%')

In [ ]:
times = pd.to_datetime(df.start_time)
dfp = df[['start_time', 'transportation_mode_tr']].pivot_table(index=times.dt.hour, columns='transportation_mode_tr', aggfunc='count')
dfp

In [ ]:
dfp.plot.bar(stacked=True)

In [ ]:
from mappymatch.constructs.geofence import Geofence
from shapely.geometry import Point
geofence = Geofence.from_geojson("sources/paris.geojson")

In [ ]:
df_in_paris = df[df.apply(lambda x: Point(x.longitude, x.latitude).within(geofence.geometry), axis=1)]

In [ ]:
import json
def filter_end(frame):
    t = json.loads(frame['trace_gps'])[-1]
    return Point(t[0], t[1]).within(geofence.geometry)
df_in_paris_end = df_in_paris[df_in_paris.apply(filter_end, axis=1)]
df_in_paris_end

In [ ]:
df_in_paris_end.groupby(['transportation_mode_tr']).size().plot.pie(autopct='%1.1f%%')

In [ ]:
df_in_paris_end[(df_in_paris_end['transportation_mode_tr'] != 'RAIL_TRIP') & (df_in_paris_end['transportation_mode_tr'] != 'FOOT')].groupby(['transportation_mode_tr']).size().plot.pie(autopct='%1.1f%%')

In [ ]:
modal_share_gps = (df_in_paris_end.groupby(['transportation_mode_tr']).size() / len(df_in_paris_end)).to_json()
modal_share_gps

In [ ]:
with open('../static/data/modal_share_gps.json', 'w') as f:
    f.write(modal_share_gps)
